In [ ]:
server_template = """
from concurrent import futures

import grpc

import {{ filename }}_pb2
import {{ filename }}_pb2_grpc

class Greeter({{ filename }}_pb2_grpc.GreeterServicer):

    def SayHello(self, request, context):
        return helloworld_pb2.HelloReply(message='Hello, %s!' % request.name)

    def SayHelloAgain(self, request, context):
        return helloworld_pb2.HelloReply(message='Hello again, %s!' % request.name)
"""

In [37]:
a = '{"arg1" : "Geeks", "arg2" : "for", "arg3" : "Geeks"}'
a.split(",")

['{"arg1" : "Geeks"', ' "arg2" : "for"', ' "arg3" : "Geeks"}']

In [1]:
import helloworld_pb2_grpc
import helloworld_pb2

In [2]:
dir(helloworld_pb2_grpc)

['Greeter',
 'GreeterServicer',
 'GreeterStub',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'add_GreeterServicer_to_server',
 'grpc',
 'helloworld__pb2']

In [1]:
import os
import sys
import glob
import subprocess

In [31]:
def readArg():
    if len(sys.argv) == 2:
        proto_file_path = str(sys.argv)[1]
        if os.path.isfile(proto_file_path):
            return proto_file_path
        else:
            raise FileNotFoundError
    else:
        print("There must be two arguments passed: \
        1. Name of the script file, \
        2. Name of the proto file")
        raise ValueError("arg not found")
    
    
def createTemplateFiles(proto_file_path):
    command_status = os.system(
        f"python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. {proto_file_path}"
    )
    if(command_status!=0):
        raise Exception(f"{proto_file_path} syntax not correct")
    


def createServerTemplate():
    server_template = """
    from concurrent import futures

    import grpc

    import {{ filename }}_pb2
    import {{ filename }}_pb2_grpc

    class Greeter({{ filename }}_pb2_grpc.GreeterServicer):

        def SayHello(self, request, context):
            return helloworld_pb2.HelloReply(message='Hello, %s!' % request.name)

        def SayHelloAgain(self, request, context):
            return helloworld_pb2.HelloReply(message='Hello again, %s!' % request.name)
    """


def createClientTemplate():
    pass


if __name__ == "__main__":
    proto_file_path = readArg()
    createTemplateFiles(proto_file_path)
    proto_file_name = proto_file_path.strip().split()[-1]
    print(proto_file_name)
    

0


In [30]:
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. helloworld.proto

In [25]:
import subprocess
subprocess.call(["ls", "-l"]).read()

AttributeError: 'int' object has no attribute 'read'

In [13]:
from pprint import pprint

In [ ]:
a = [5,5,3,5,6,3,7,9,2,1,3]

In [6]:
from pyparsing import (Word, alphas, alphanums, Regex, Suppress, Forward,
    Group, oneOf, ZeroOrMore, Optional, delimitedList, Keyword,
    restOfLine, quotedString, Dict)

ident = Word(alphas+"_",alphanums+"_").setName("identifier")
integer = Regex(r"[+-]?\d+")

LBRACE,RBRACE,LBRACK,RBRACK,LPAR,RPAR,EQ,SEMI = map(Suppress,"{}[]()=;")

kwds = """message required optional repeated enum extensions extends extend 
          to package service rpc returns true false option import"""
for kw in kwds.split():
    exec("%s_ = Keyword('%s')" % (kw.upper(), kw))

messageBody = Forward()

messageDefn = MESSAGE_ - ident("messageId") + LBRACE + messageBody("body") + RBRACE

typespec = oneOf("""double float int32 int64 uint32 uint64 sint32 sint64 
                    fixed32 fixed64 sfixed32 sfixed64 bool string bytes""") | ident
rvalue = integer | TRUE_ | FALSE_ | ident
fieldDirective = LBRACK + Group(ident + EQ + rvalue) + RBRACK
fieldDefn = (( REQUIRED_ | OPTIONAL_ | REPEATED_ )("fieldQualifier") - 
              typespec("typespec") + ident("ident") + EQ + integer("fieldint") + ZeroOrMore(fieldDirective) + SEMI)

# enumDefn ::= 'enum' ident '{' { ident '=' integer ';' }* '}'
enumDefn = ENUM_("typespec") - ident('name') + LBRACE + Dict( ZeroOrMore( Group(ident + EQ + integer + SEMI) ))('values') + RBRACE

# extensionsDefn ::= 'extensions' integer 'to' integer ';'
extensionsDefn = EXTENSIONS_ - integer + TO_ + integer + SEMI

# messageExtension ::= 'extend' ident '{' messageBody '}'
messageExtension = EXTEND_ - ident + LBRACE + messageBody + RBRACE

# messageBody ::= { fieldDefn | enumDefn | messageDefn | extensionsDefn | messageExtension }*
messageBody << Group(ZeroOrMore( Group(fieldDefn | enumDefn | messageDefn | extensionsDefn | messageExtension) ))

# methodDefn ::= 'rpc' ident '(' [ ident ] ')' 'returns' '(' [ ident ] ')' ';'
methodDefn = (RPC_ - ident("methodName") + 
              LPAR + Optional(ident("methodParam")) + RPAR + 
              RETURNS_ + LPAR + Optional(ident("methodReturn")) + RPAR)

# serviceDefn ::= 'service' ident '{' methodDefn* '}'
serviceDefn = SERVICE_ - ident("serviceName") + LBRACE + ZeroOrMore(Group(methodDefn)) + RBRACE

# packageDirective ::= 'package' ident [ '.' ident]* ';'
packageDirective = Group(PACKAGE_ - delimitedList(ident, '.', combine=True) + SEMI)

comment = '//' + restOfLine

importDirective = IMPORT_ - quotedString("importFileSpec") + SEMI

optionDirective = OPTION_ - ident("optionName") + EQ + quotedString("optionValue") + SEMI

topLevelStatement = Group(messageDefn | messageExtension | enumDefn | serviceDefn | importDirective | optionDirective)

parser = Optional(packageDirective) + ZeroOrMore(topLevelStatement)

parser.ignore(comment)

{[Group:({{{"package" -} identifier [. identifier]...} Suppress:(";")})] [Group:({{{{{{{{{{"message" -} identifier} Suppress:("{")} Forward: Forward: Group:([Group:({{{{{{{{{{{{{"required" | "optional"} | "repeated"} -} {double | float | int32 | int64 | uint32 | uint64 | sint32 | sint64 | fixed32 | fixed64 | sfixed32 | sfixed64 | bool | string | bytes | identifier}} identifier} Suppress:("=")} Re:('[+-]?\\d+')} [{{Suppress:("[") Group:({{identifier Suppress:("=")} {{{Re:('[+-]?\\d+') | "true"} | "false"} | identifier}})} Suppress:("]")}]...} Suppress:(";")} | {{{{{"enum" -} identifier} Suppress:("{")} Dict:([Group:({{{identifier Suppress:("=")} Re:('[+-]?\\d+')} Suppress:(";")})]...)} Suppress:("}")}} | {{{{{"message" -} identifier} Suppress:("{")} : ...} Suppress:("}")}} | {{{{{"extensions" -} Re:('[+-]?\\d+')} "to"} Re:('[+-]?\\d+')} Suppress:(";")}} | {{{{{"extend" -} identifier} Suppress:("{")} : ...} Suppress:("}")}})]...)} Suppress:("}")} | {{{{{"extend" -} identifier} Suppress:(

In [7]:
test1 = """message Person { 
  required int32 id = 1; 
  required string name = 2; 
  optional string email = 3; 
}"""

test2 = """package tutorial;

message Person {
  required string name = 1;
  required int32 id = 2;
  optional string email = 3;

  enum PhoneType {
    MOBILE = 0;
    HOME = 1;
    WORK = 2;
  }

  message PhoneNumber {
    required string number = 1;
    optional PhoneType type = 2 [default = HOME];
  }

  repeated PhoneNumber phone = 4;
}

message AddressBook {
  repeated Person person = 1;
}"""

In [26]:
out = parser.runTests([test1])


message Person { 
  required int32 id = 1; 
  required string name = 2; 
  optional string email = 3; 
}
[['message', 'Person', [['required', 'int32', 'id', '1'], ['required', 'string', 'name', '2'], ['optional', 'string', 'email', '3']]]]
[0]:
  ['message', 'Person', [['required', 'int32', 'id', '1'], ['required', 'string', 'name', '2'], ['optional', 'string', 'email', '3']]]
  - body: [['required', 'int32', 'id', '1'], ['required', 'string', 'name', '2'], ['optional', 'string', 'email', '3']]
    [0]:
      ['required', 'int32', 'id', '1']
      - fieldQualifier: 'required'
      - fieldint: '1'
      - ident: 'id'
      - typespec: 'int32'
    [1]:
      ['required', 'string', 'name', '2']
      - fieldQualifier: 'required'
      - fieldint: '2'
      - ident: 'name'
      - typespec: 'string'
    [2]:
      ['optional', 'string', 'email', '3']
      - fieldQualifier: 'optional'
      - fieldint: '3'
      - ident: 'email'
      - typespec: 'string'
  - messageId: 'Person'


In [27]:
pprint(out)

(True,
 [('message Person { \n'
   '  required int32 id = 1; \n'
   '  required string name = 2; \n'
   '  optional string email = 3; \n'
   '}',
   ([(['message', 'Person', ([(['required', 'int32', 'id', '1'], {'fieldQualifier': ['required'], 'typespec': ['int32'], 'ident': ['id'], 'fieldint': ['1']}), (['required', 'string', 'name', '2'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['name'], 'fieldint': ['2']}), (['optional', 'string', 'email', '3'], {'fieldQualifier': ['optional'], 'typespec': ['string'], 'ident': ['email'], 'fieldint': ['3']})], {})], {'messageId': ['Person'], 'body': [([(['required', 'int32', 'id', '1'], {'fieldQualifier': ['required'], 'typespec': ['int32'], 'ident': ['id'], 'fieldint': ['1']}), (['required', 'string', 'name', '2'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['name'], 'fieldint': ['2']}), (['optional', 'string', 'email', '3'], {'fieldQualifier': ['optional'], 'typespec': ['string'], 'ident': ['email'],

In [28]:
out[1][0][1][0]

(['message', 'Person', ([(['required', 'int32', 'id', '1'], {'fieldQualifier': ['required'], 'typespec': ['int32'], 'ident': ['id'], 'fieldint': ['1']}), (['required', 'string', 'name', '2'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['name'], 'fieldint': ['2']}), (['optional', 'string', 'email', '3'], {'fieldQualifier': ['optional'], 'typespec': ['string'], 'ident': ['email'], 'fieldint': ['3']})], {})], {'messageId': ['Person'], 'body': [([(['required', 'int32', 'id', '1'], {'fieldQualifier': ['required'], 'typespec': ['int32'], 'ident': ['id'], 'fieldint': ['1']}), (['required', 'string', 'name', '2'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['name'], 'fieldint': ['2']}), (['optional', 'string', 'email', '3'], {'fieldQualifier': ['optional'], 'typespec': ['string'], 'ident': ['email'], 'fieldint': ['3']})], {})]})

In [25]:
out2 = parser.runTests([test2])


package tutorial;

message Person {
  required string name = 1;
  required int32 id = 2;
  optional string email = 3;

  enum PhoneType {
    MOBILE = 0;
    HOME = 1;
    WORK = 2;
  }

  message PhoneNumber {
    required string number = 1;
    optional PhoneType type = 2 [default = HOME];
  }

  repeated PhoneNumber phone = 4;
}

message AddressBook {
  repeated Person person = 1;
}
[['package', 'tutorial'], ['message', 'Person', [['required', 'string', 'name', '1'], ['required', 'int32', 'id', '2'], ['optional', 'string', 'email', '3'], ['enum', 'PhoneType', [['MOBILE', '0'], ['HOME', '1'], ['WORK', '2']]], ['message', 'PhoneNumber', [['required', 'string', 'number', '1'], ['optional', 'PhoneType', 'type', '2', ['default', 'HOME']]]], ['repeated', 'PhoneNumber', 'phone', '4']]], ['message', 'AddressBook', [['repeated', 'Person', 'person', '1']]]]
[0]:
  ['package', 'tutorial']
[1]:
  ['message', 'Person', [['required', 'string', 'name', '1'], ['required', 'int32', 'id', '2'], ['op

In [29]:
pprint(out2)

(True,
 [('package tutorial;\n'
   '\n'
   'message Person {\n'
   '  required string name = 1;\n'
   '  required int32 id = 2;\n'
   '  optional string email = 3;\n'
   '\n'
   '  enum PhoneType {\n'
   '    MOBILE = 0;\n'
   '    HOME = 1;\n'
   '    WORK = 2;\n'
   '  }\n'
   '\n'
   '  message PhoneNumber {\n'
   '    required string number = 1;\n'
   '    optional PhoneType type = 2 [default = HOME];\n'
   '  }\n'
   '\n'
   '  repeated PhoneNumber phone = 4;\n'
   '}\n'
   '\n'
   'message AddressBook {\n'
   '  repeated Person person = 1;\n'
   '}',
   ([(['package', 'tutorial'], {}), (['message', 'Person', ([(['required', 'string', 'name', '1'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['name'], 'fieldint': ['1']}), (['required', 'int32', 'id', '2'], {'fieldQualifier': ['required'], 'typespec': ['int32'], 'ident': ['id'], 'fieldint': ['2']}), (['optional', 'string', 'email', '3'], {'fieldQualifier': ['optional'], 'typespec': ['string'], 'ident': ['email

In [33]:
out2[1][0][1][1]

(['message', 'Person', ([(['required', 'string', 'name', '1'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['name'], 'fieldint': ['1']}), (['required', 'int32', 'id', '2'], {'fieldQualifier': ['required'], 'typespec': ['int32'], 'ident': ['id'], 'fieldint': ['2']}), (['optional', 'string', 'email', '3'], {'fieldQualifier': ['optional'], 'typespec': ['string'], 'ident': ['email'], 'fieldint': ['3']}), (['enum', 'PhoneType', ([(['MOBILE', '0'], {}), (['HOME', '1'], {}), (['WORK', '2'], {})], {'MOBILE': ['0'], 'HOME': ['1'], 'WORK': ['2']})], {'typespec': ['enum'], 'name': ['PhoneType'], 'values': [([(['MOBILE', '0'], {}), (['HOME', '1'], {}), (['WORK', '2'], {})], {'MOBILE': ['0'], 'HOME': ['1'], 'WORK': ['2']})]}), (['message', 'PhoneNumber', ([(['required', 'string', 'number', '1'], {'fieldQualifier': ['required'], 'typespec': ['string'], 'ident': ['number'], 'fieldint': ['1']}), (['optional', 'PhoneType', 'type', '2', (['default', 'HOME'], {})], {'fieldQualifier'

In [167]:
test3 = """// Copyright 2015 gRPC authors.
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//     http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.





package helloworld;

// The request message containing the user's name.
message HelloRequest {
  required string name = 1;
}

// The response message containing the greetings
message HelloReply {
  required string message = 1;
  required int32 result = 2;
}"""

In [168]:
out3 = parser.runTests([test3])


// Copyright 2015 gRPC authors.
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//     http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.





package helloworld;

// The request message containing the user's name.
message HelloRequest {
  required string name = 1;
}

// The response message containing the greetings
message HelloReply {
  required string message = 1;
  required int32 result = 2;
}
[['package', 'helloworld'], ['message', 'HelloRequest', [['required', 'string', 'name', '1']]], ['message', 'HelloReply', [['required', 'string', 

In [194]:
out3[1][0][1][1][2][0]["ident"]

'name'

In [37]:
import helloworld_pb2 as h

In [80]:
h.DESCRIPTOR.message_types_by_name.keys()

['HelloRequest', 'HelloReply']

In [59]:
h.DESCRIPTOR.services_by_name.keys()

['Greeter']

In [71]:
import helloworld_pb2_grpc as hg

In [74]:
dir(hg.Greeter)

['SayHello',
 'SayHelloAgain',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [81]:
dir(h.HelloRequest)

['ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DESCRIPTOR',
 'DiscardUnknownFields',
 'Extensions',
 'FindInitializationErrors',
 'FromString',
 'HasExtension',
 'HasField',
 'IsInitialized',
 'ListFields',
 'MergeFrom',
 'MergeFromString',
 'ParseFromString',
 'RegisterExtension',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'UnknownFields',
 'WhichOneof',
 '_CheckCalledFromGeneratedFile',
 '_SetListener',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_extensions_by_name',
 '_extensions_by_number',
 'name']

In [46]:
dir(h.DESCRIPTOR)

['CopyToProto',
 'GetOptions',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_options',
 '_serialized_options',
 'dependencies',
 'enum_types_by_name',
 'extensions_by_name',
 'has_options',
 'message_types_by_name',
 'name',
 'package',
 'pool',
 'public_dependencies',
 'serialized_pb',
 'services_by_name',
 'syntax']

In [51]:
from pprint import pprint
pprint(vars(h.DESCRIPTOR))

TypeError: vars() argument must have __dict__ attribute

In [143]:
import re

with open(f"/home/tushar/file/grpc/test_script/helloworld.proto", 'r') as r:
        file_text = r.read()
        text = re.findall('service .*? \{.*\}', file_text, re.S)[0]
        count = None
        newtext = ''
        for i, char in enumerate(text):
            def checkCount(count):
                if count == None:
                    count = 0
                return count
                
            if char == '{':
                count = checkCount(count)
                count = count + 1
                print(count)
            elif char == '}':
                count = checkCount(count)
                count = count - 1
                print(count)
            if count == 0:
                newtext = text[:i+1]
                break
        print(newtext)

1
2
1
2
1
0
service Greeter {
  // Sends a greeting
  rpc SayHello (HelloRequest) returns (HelloReply) {}
  rpc SayHelloAgain (HelloRequest) returns (HelloReply) {}
}


In [219]:
import re

with open(f"/home/tushar/file/grpc/test_script/helloworld.proto", 'r') as r:
    rpc_dict = {
        "request" : {
            "stream": False,
            "func": "",
            "message_name": "",
            "message_param": {}
        },
        "response": {
            "stream": False,
            "func": "",
            "message_name": "",
            "message_param": {}
        }
    }
    file_text = r.read()
    text = re.findall('rpc .*?\{.*?\}?', file_text, re.S)
    print(text)
    for sentence in text:
        
        print(sentence.split(' '))

['rpc SayHello (HelloRequest) returns (HelloReply) {}', 'rpc SayHelloAgain (HelloRequest) returns (HelloReply) {}']
['rpc', 'SayHello', '(HelloRequest)', 'returns', '(HelloReply)', '{}']
['rpc', 'SayHelloAgain', '(HelloRequest)', 'returns', '(HelloReply)', '{}']


In [241]:
import re

with open(f"/home/tushar/file/grpc/test_script/helloworld.proto", 'r') as r:
    rpc_dict = {
        "func": "",
        "stream_req": False,
        "req_message_name": "",
        "req_message_param": [],
        "stream_res": False,
        "res_message_name": "",
        "res_message_param": []
    }
    file_text = r.read()
    without_comments_text = re.sub(r'//.*\n', '', file_text)
    rpc_text = re.findall('rpc .*?\{.*?\}?', file_text, re.S)
    
    for sentence in rpc_text:
        sentence = sentence.split(" ")
        rpc_dict["func"] = sentence[1]
        rpc_dict["req_message_name"] = sentence[2]
        rpc_dict["res_message_name"] = sentence[4]
    
    message_text = re.findall('message .*?\{.*?\}', without_comments_text, re.S)
    for sentence in message_text:
        message_text = sentence.replace("\n", '').split("{")
        messageBody = re.findall('\S+', message_text[0].strip())[-1]
        message_text = re.findall('\S+', message_text[1].strip())
        for i in range(len(message_text)):
            if rpc_dict.get("req_message_name") == message_text[1]:
                if message_text[i] == "=":
                    rpc_dict["req_message_param"].append(message_text[i-1])
            elif rpc_dict.get("res_message_name") == message_text[1]:
                if message_text[i] == "=":
                    rpc_dict["res_message_param"].append(message_text[i-1])
            else:
                pass
print(rpc_dict)        

{'func': 'SayHelloAgain', 'stream_req': False, 'req_message_name': '(HelloRequest)', 'req_message_param': [], 'stream_res': False, 'res_message_name': '(HelloReply)', 'res_message_param': []}


In [117]:

def check(my_string): 
    brackets = ['()', '{}', '[]']
    while any(x in my_string for x in brackets): 
        for br in brackets: 
            my_string = my_string.replace(br, '') 
    return not my_string

In [122]:
string = "{(}"
print(string, "-", "Balanced" 
if check(string) else "Unbalanced") 

{(} - Unbalanced
